In [66]:
import requests
import pprint
import json
from datetime import datetime

In [15]:
headers = {
	"X-RapidAPI-Key": "1b0a42ee93msh3c60044810c2171p15d22bjsn431cea4eea20",
	"X-RapidAPI-Host": "api-football-v1.p.rapidapi.com"
}

pp = pprint.PrettyPrinter(depth=6)

In [25]:
# Finding all leagues in England
url = "https://api-football-v1.p.rapidapi.com/v3/leagues"

querystring = {"country":"England"}

response = requests.request("GET", url, headers=headers, params=querystring)

loaded_r = json.loads(response.text)

with open('result.json', 'w') as fp:
    json.dump(loaded_r, fp)

In [50]:
# Getting all teams in Premier League
league_id = 2
url = "https://api-football-v1.p.rapidapi.com/v2/teams/league/" + str(league_id)

response = requests.request("GET", url, headers=headers)

loaded_r = json.loads(response.text)

with open('premier_league.json', 'w') as fp:
    json.dump(loaded_r, fp)

In [108]:
def get_key_observations(team1, team2, month, day, year):
    # get fixture id of mentioned match
    url = "https://api-football-v1.p.rapidapi.com/v3/fixtures/headtohead"
    
    with open('team_to_id.json', 'r') as f:
        data = json.load(f)
        
        team1_id = data[team1]["team_id"]
        team2_id = data[team2]["team_id"]
        
        h2h = "{id1}-{id2}".format(id1=team1_id, id2=team2_id)
        date = f'{year:02}-{month:02}-{day:02}' 
        # print(date)
        
    querystring = {"h2h":h2h, "date":date}

    response = requests.request("GET", url, headers=headers, params=querystring)
    loaded_r = json.loads(response.text)
    
    for fixture in loaded_r["response"]:
        fixture_id = fixture["fixture"]["id"]
        score_dict = fixture["score"]
        home_away = {"home": fixture["teams"]["home"]["name"], "away": fixture["teams"]["away"]["name"]}
        # print(home_away)
    
    # get fixture statistics 
    url = "https://api-football-v1.p.rapidapi.com/v3/fixtures/statistics"
    
    querystring = {"fixture":fixture_id}
    
    response = requests.request("GET", url, headers=headers, params=querystring)
    loaded_r = json.loads(response.text)
    
    # with open('temp_stats.json', 'w') as fp:
    #     json.dump(loaded_r, fp)
    
    curr_dict = {}
    
    for team in loaded_r["response"]:
        name = team["team"]["name"]

        stat = team["statistics"]
        temp = {}
        
        # add statistics to dict
        for curr in stat:
            curr_type, val = curr["type"], curr["value"]
            temp[curr_type] = val
            
        curr_dict[name] = temp
    # print(curr_dict)
        
    # add halftime and fulltime score to statistics
    for ele in score_dict:
        home_score = score_dict[ele]["home"]
        away_score = score_dict[ele]["away"]

        home_team = home_away["home"]
        away_team = home_away["away"]

        print(ele, home_team, away_team)

        curr_dict[home_team][ele] = home_score
        curr_dict[away_team][ele] = away_score
    
    return curr_dict

        
get_key_observations("Manchester United", "Newcastle", 4, 2, 2023)

{'Newcastle': {'Shots on Goal': 6, 'Shots off Goal': 9, 'Total Shots': 22, 'Blocked Shots': 7, 'Shots insidebox': 16, 'Shots outsidebox': 6, 'Fouls': 8, 'Corner Kicks': 7, 'Offsides': 1, 'Ball Possession': '46%', 'Yellow Cards': None, 'Red Cards': None, 'Goalkeeper Saves': 1, 'Total passes': 343, 'Passes accurate': 264, 'Passes %': '77%', 'expected_goals': '3.68'}, 'Manchester United': {'Shots on Goal': 1, 'Shots off Goal': 3, 'Total Shots': 6, 'Blocked Shots': 2, 'Shots insidebox': 5, 'Shots outsidebox': 1, 'Fouls': 12, 'Corner Kicks': 4, 'Offsides': None, 'Ball Possession': '54%', 'Yellow Cards': None, 'Red Cards': None, 'Goalkeeper Saves': 4, 'Total passes': 417, 'Passes accurate': 338, 'Passes %': '81%', 'expected_goals': '0.43'}}
halftime Newcastle Manchester United
fulltime Newcastle Manchester United
extratime Newcastle Manchester United
penalty Newcastle Manchester United
{'Newcastle': {'Shots on Goal': 6, 'Shots off Goal': 9, 'Total Shots': 22, 'Blocked Shots': 7, 'Shots insid

In [52]:
# Create another json from premier_league with team name as key, val as {name, country}

curr = {}

with open('premier_league.json', 'r') as f:
    data = json.load(f)

    for curr_team in data["api"]["teams"]:
        # print(curr_team)
        team_id = curr_team["team_id"]
        name = curr_team["name"]
        
        curr[name] = {"team_id": team_id}
        
with open('team_to_id.json', 'w') as f:
    json.dump(curr, f)
        

In [97]:
# Test head to head endpoint
querystring = {"h2h":"33-34", "date": "2023-04-02"}

url = "https://api-football-v1.p.rapidapi.com/v3/fixtures/headtohead"

response = requests.request("GET", url, headers=headers, params=querystring)

pp.pprint(response.text)

('{"get":"fixtures\\/headtohead","parameters":{"date":"2023-04-02","h2h":"33-34"},"errors":[],"results":1,"paging":{"current":1,"total":1},"response":[{"fixture":{"id":868233,"referee":"Stuart '
 'Attwell, '
 'England","timezone":"UTC","date":"2023-04-02T15:30:00+00:00","timestamp":1680449400,"periods":{"first":1680449400,"second":1680453000},"venue":{"id":562,"name":"St. '
 'James\' Park","city":"Newcastle upon Tyne"},"status":{"long":"Match '
 'Finished","short":"FT","elapsed":90}},"league":{"id":39,"name":"Premier '
 'League","country":"England","logo":"https:\\/\\/media-3.api-sports.io\\/football\\/leagues\\/39.png","flag":"https:\\/\\/media-2.api-sports.io\\/flags\\/gb.svg","season":2022,"round":"Regular '
 'Season - '
 '29"},"teams":{"home":{"id":34,"name":"Newcastle","logo":"https:\\/\\/media-2.api-sports.io\\/football\\/teams\\/34.png","winner":true},"away":{"id":33,"name":"Manchester '
 'United","logo":"https:\\/\\/media-3.api-sports.io\\/football\\/teams\\/33.png","winner":fa